## Goal driven error estimator:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
import numpy as np

In [ ]:
def DoerflerMarking(err, theta=0.25):
    sort_ind = np.flip(np.argsort(err.NumPy()))
    v = theta*sum(err)
    
    refine_val = 0
    marking_flag = np.zeros(len(err))
    for i in range(len(err)):
        refine_val += err[sort_ind[i]]
        marking_flag[sort_ind[i]] = True
        if v < refine_val:
            break
            
    return marking_flag

In [ ]:
maxh = 1

from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
cos = [(0.7,0.45), (0.75,0.45), (0.8,0.45), (0.8, 0.55), (0.75,0.55), (0.7,0.55), (0.75,0.5)]
for co in cos:
    geo.AddPoint(*co)
geo.Append(["line", 0,1], leftdomain=3, rightdomain=1, bc='none'  , maxh=maxh)
geo.Append(["line", 1,6], leftdomain=3, rightdomain=3, bc='goal_2', maxh=maxh)
geo.Append(["line", 6,4], leftdomain=3, rightdomain=3, bc='goal_2', maxh=maxh)
geo.Append(["line", 4,5], leftdomain=3, rightdomain=1, bc='none'  , maxh=maxh)
geo.Append(["line", 5,0], leftdomain=3, rightdomain=1, bc='none'  , maxh=maxh)
geo.Append(["line", 1,2], leftdomain=3, rightdomain=1, bc='none'  , maxh=maxh)
geo.Append(["line", 2,3], leftdomain=3, rightdomain=1, bc='none'  , maxh=maxh)
geo.Append(["line", 3,4], leftdomain=3, rightdomain=1, bc='none'  , maxh=maxh)
geo.AddRectangle( (0,0), (1,1), leftdomain=1, rightdomain=0, bc='outer')
geo.AddRectangle( (0.2,0.45), (0.3,0.55), leftdomain=2, rightdomain=1,maxh=0.05, 
                 bcs=['inner_bot', 'inner_right', 'inner_top', 'inner_left'])
geo.SetMaterial(1, 'outer')
geo.SetMaterial(2, 'source')
geo.SetMaterial(3, 'goal_1')

mesh = Mesh(geo.GenerateMesh(maxh=maxh))
Draw(CoefficientFunction( [1,2,3] ), mesh, "domains")

In [ ]:
def MethodZZ(order, modus, maxndof=10000):
    mesh = Mesh(geo.GenerateMesh(maxh=1))
    fes = H1(mesh, order=order, dirichlet="outer", autoupdate=True)    
    u,v = fes.TnT()

    a = BilinearForm(fes, symmetric=True)
    a += grad(u)*grad(v)*dx

    f = LinearForm(fes)
    f += 100*v*dx("source")
    
    b = LinearForm(fes)
    if modus == 0:
        b += 100*v*dx("goal_1")
    elif modus == 1:
        b += 10*v*ds("goal_2")
    else:
        b += v(0.75,0.5)

    gfu = GridFunction(fes, autoupdate=True)
    space_flux = VectorH1(mesh, order=order, autoupdate=True)
    gf_flux    = GridFunction(space_flux, autoupdate=True)
    
    def SolveBVP():
        a.Assemble()
        f.Assemble()
        b.Assemble()
        gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
        return (fes.ndof, InnerProduct(b.vec,gfu.vec))
    
    def CalcError():
        flux = grad(gfu)
        gf_flux.Set(flux)

        err = (flux-gf_flux)*(flux-gf_flux)
        elerr = Integrate (err, mesh, VOL, element_wise=True)

        marking_flag = DoerflerMarking(elerr)
    
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, marking_flag[el.nr])
        
        return
    
    result = []
    with TaskManager():
        while fes.ndof < maxndof:
            result.append(SolveBVP())
            CalcError()
            mesh.Refine()
    result.append(SolveBVP())
    #Draw(gfu, mesh, "u")
    return result

In [ ]:
def MethodGD(order, modus, maxndof=10000):
    mesh = Mesh(geo.GenerateMesh(maxh=1))
    fes = H1(mesh, order=order, dirichlet="outer", autoupdate=True)    
    u,v = fes.TnT()

    a = BilinearForm(fes, symmetric=True)
    a += grad(u)*grad(v)*dx

    f = LinearForm(fes)
    f += 100*v*dx("source")
    
    b = LinearForm(fes)
    if modus == 0:
        b += 100*v*dx("goal_1")
    elif modus == 1:
        b += 10*v*ds("goal_2")
    else:
        b += v(0.75,0.5)

    gfu = GridFunction(fes, autoupdate=True)
    gfw = GridFunction(fes, autoupdate=True)
    space_flux   = VectorH1(mesh, order=order, autoupdate=True)
    gf_flux      = GridFunction(space_flux, autoupdate=True)
    gf_flux_dual = GridFunction(space_flux, autoupdate=True)
     
    def SolveBVP():
        a.Assemble()
        f.Assemble()
        b.Assemble()
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        gfu.vec.data = inv*f.vec
        gfw.vec.data = inv*b.vec
        return (fes.ndof, InnerProduct(b.vec,gfu.vec))
    
    def CalcError():
        flux = grad(gfu)
        gf_flux.Set(flux)

        err = (flux-gf_flux)*(flux-gf_flux)
        elerr = Integrate (err, mesh, VOL, element_wise=True)

        flux_dual = grad(gfw)
        gf_flux_dual.Set(flux_dual)

        err_dual = (flux_dual-gf_flux_dual)*(flux_dual-gf_flux_dual)
        elerr_dual = Integrate (err_dual, mesh, VOL, element_wise=True)

        for i in range(len(elerr)):
            elerr[i] *= elerr_dual[i]
            
        marking_flag = DoerflerMarking(elerr)
    
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, marking_flag[el.nr])
        
        return
    
    result = []
    with TaskManager():
        while fes.ndof < maxndof:
            result.append(SolveBVP())
            CalcError()
            mesh.Refine()
    result.append(SolveBVP())
    #Draw(gfu, mesh, "u")
    return result

In [ ]:
refvalues = [0.042556207995730,0.042349426604237,0.042557119266960]

In [ ]:
orders = [1,2,3]
maxndof = 15000
result_gd_m0 = []
result_gd_m1 = []
result_gd_m2 = []

result_zz_m0 = []
result_zz_m1 = []
result_zz_m2 = []

for k in orders:
    result_gd_m0.append(MethodGD(k,0,maxndof))
    result_gd_m1.append(MethodGD(k,1,maxndof))
    result_gd_m2.append(MethodGD(k,2,maxndof))
    
for k in orders:
    result_zz_m0.append(MethodZZ(k,0,maxndof))
    result_zz_m1.append(MethodZZ(k,1,maxndof))
    result_zz_m2.append(MethodZZ(k,2,maxndof))

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import interactive

plt.figure(1)
for k in range(len(orders)):
    ndof,values = zip(*result_gd_m0[k])
    err = [ abs(val-refvalues[0])/refvalues[0] for val in values]
    plt.plot(ndof, err, '-x', label="gd,mod=0,k="+str(orders[k]))

for k in range(len(orders)):
    ndof,values = zip(*result_gd_m1[k])
    err = [ abs(val-refvalues[1])/refvalues[1] for val in values]
    plt.plot(ndof, err, '-o', label="gd,mod=1,k="+str(orders[k]))

for k in range(len(orders)):
    ndof,values = zip(*result_gd_m2[k])
    err = [ abs(val-refvalues[2])/refvalues[2] for val in values]
    plt.plot(ndof, err, '-*', label="gd,mod=2,k="+str(orders[k]))
    
plt.plot(ndof, [1/sqrt(n)**2/10 for n in ndof], '-', label="$O(h^2)$")
plt.plot(ndof, [1/sqrt(n)**3/10 for n in ndof], '-', label="$O(h^3)$")
plt.plot(ndof, [1/sqrt(n)**4/10 for n in ndof], '-', label="$O(h^4)$")

plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")

plt.ylabel("error")
plt.legend()
plt.show()
interactive(True)

In [ ]:
plt.figure(2)
for k in range(len(orders)):
    ndof,values = zip(*result_zz_m0[k])
    err = [ abs(val-refvalues[0])/refvalues[0] for val in values]
    plt.plot(ndof, err, '-x', label="zz,mod=0,k="+str(orders[k]))

for k in range(len(orders)):
    ndof,values = zip(*result_zz_m1[k])
    err = [ abs(val-refvalues[1])/refvalues[1] for val in values]
    plt.plot(ndof, err, '-o', label="zz,mod=1,k="+str(orders[k]))

for k in range(len(orders)):
    ndof,values = zip(*result_zz_m2[k])
    err = [ abs(val-refvalues[2])/refvalues[2] for val in values]
    plt.plot(ndof, err, '-*', label="zz,mod=2,k="+str(orders[k]))

plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")

plt.ylabel("error")
plt.legend()
plt.show()
interactive(True)